Unzip the files from google drive

In [1]:
!unzip /content/drive/MyDrive/shout-data.zip

Streaming output truncated to the last 5000 lines.
   creating: shout-data/shout_data_8f1bd9f1-9754-46c3-bd5b-fa9b0cf3acc1/
   creating: shout-data/shout_data_8f1bd9f1-9754-46c3-bd5b-fa9b0cf3acc1/final/
  inflating: shout-data/shout_data_8f1bd9f1-9754-46c3-bd5b-fa9b0cf3acc1/final/chunk10.wav  
  inflating: shout-data/shout_data_8f1bd9f1-9754-46c3-bd5b-fa9b0cf3acc1/final/chunk20.wav  
  inflating: shout-data/shout_data_8f1bd9f1-9754-46c3-bd5b-fa9b0cf3acc1/final/chunk30.wav  
  inflating: shout-data/shout_data_8f1bd9f1-9754-46c3-bd5b-fa9b0cf3acc1/final/chunk31.wav  
  inflating: shout-data/shout_data_8f1bd9f1-9754-46c3-bd5b-fa9b0cf3acc1/final/chunk40.wav  
  inflating: shout-data/shout_data_8f1bd9f1-9754-46c3-bd5b-fa9b0cf3acc1/final/chunk50.wav  
   creating: shout-data/shout_data_8f280ac7-c191-4681-b683-95f5ad1f25a2/
   creating: shout-data/shout_data_8f280ac7-c191-4681-b683-95f5ad1f25a2/final/
  inflating: shout-data/shout_data_8f280ac7-c191-4681-b683-95f5ad1f25a2/final/chunk10.wav  
 

install the libraries needed


In [ ]:
!pip install pandas numpy matplotlib scikit-learn tensorflow librosa

In [3]:
import os
import numpy as np
import librosa
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Reshape, LSTM, Bidirectional,  ELU, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam




loading the files needed

In [4]:
#file paths
csv_data = "/content/drive/MyDrive/test_data.csv"
filtered_data = "/content/drive/MyDrive/filtered_training_data.csv"
# reads the csv
training_data = pd.read_csv(filtered_data)
# reads the csv
test_data = pd.read_csv(csv_data)
# Load the data
training_data = pd.read_csv(filtered_data)
test_data = pd.read_csv(csv_data)

mfcc function and fix the file pathing in the csv

In [5]:
def extract_mfcc(file_path, n_fft=2048, hop_length=512, max_sequence_length = 680):
    # load audio file and sampling rate
    y, sr = librosa.load(file_path, sr=None)

    # Normalize the audio data
    y = y / np.max(np.abs(y))

    # extract MFCC features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)

    # normalize the features
    # mfccs = (mfccs - np.mean(mfccs)) / np.std(mfccs)
    mfccs = (mfccs - np.mean(mfccs, axis=1, keepdims=True)) / np.std(mfccs, axis=1, keepdims=True)

    if max_sequence_length is not None:
        if mfccs.shape[1] < max_sequence_length:
            mfccs = np.pad(mfccs, ((0, 0), (0, max_sequence_length - mfccs.shape[1])), mode='constant')
        elif mfccs.shape[1] > max_sequence_length:
            mfccs = mfccs[:, :max_sequence_length]

    return mfccs


mfcc training

In [6]:
# remove the .wav in file_name and combine it with chunk so it can read the path in the folders
training_data.loc[:, 'file_name'] = training_data['file_name'].str.replace('/media/data/', '/content/').str.replace('.wav', '')
csv_combined_wave_path = training_data['file_name'] + training_data['chunk_name']

# all the wave files in the csv
csv_combined_wave_path = csv_combined_wave_path.tolist()
print("size of all paths to train: ", len(csv_combined_wave_path), csv_combined_wave_path)

# use a dictionary to store the csv path and the wave path since they are different when using mounted google drive
path_dict = {}
for i in range(len(csv_combined_wave_path)):
  path_dict[csv_combined_wave_path[i]] = csv_combined_wave_path[i]
print(path_dict)


mfcc_train = []


mfcc_train = [extract_mfcc(path_dict[file_path]) for file_path in csv_combined_wave_path]

# can use mfcc_train to train model below
mfcc_train = np.array(mfcc_train)

print("Training MFCC size: ", len(mfcc_train), ", shape:", mfcc_train.shape, "sequence: ", mfcc_train)

np.save('mfccs_train.npy', mfcc_train)



size of all paths to train:  2953 ['/content/shout-data/shout_data_8b12154a-ac79-4560-b108-9bde561b8027/final/chunk10.wav', '/content/shout-data/shout_data_8b12154a-ac79-4560-b108-9bde561b8027/final/chunk20.wav', '/content/shout-data/shout_data_4b02236c-4083-45cc-8ad2-a016673ee58c/final/chunk11.wav', '/content/shout-data/shout_data_4b02236c-4083-45cc-8ad2-a016673ee58c/final/chunk12.wav', '/content/shout-data/shout_data_4b02236c-4083-45cc-8ad2-a016673ee58c/final/chunk13.wav', '/content/shout-data/shout_data_ec347b5a-18ff-41bc-b5e2-c02aa55abf60/final/chunk11.wav', '/content/shout-data/shout_data_ec347b5a-18ff-41bc-b5e2-c02aa55abf60/final/chunk31.wav', '/content/shout-data/shout_data_ec347b5a-18ff-41bc-b5e2-c02aa55abf60/final/chunk40.wav', '/content/shout-data/shout_data_6df385ea-b7ee-44c6-ad1d-7ea3836854cb/final/chunk30.wav', '/content/shout-data/shout_data_e09a0211-1622-46c0-81e1-43de2d73d09c/final/chunk01.wav', '/content/shout-data/shout_data_e09a0211-1622-46c0-81e1-43de2d73d09c/final/

mfcc test

In [7]:
# get the path for wav file
test_data.loc[:, 'file_name'] = test_data['file_name'].str.replace('/media/data/', '/content/').str.replace('.wav', '')
csv_combined_wave_path = test_data['file_name'] + test_data['chunk_name']
# print("combined wave paths before to list: ", csv_combined_wave_path)

# all the wave files in the csv
csv_combined_wave_path = csv_combined_wave_path.tolist()
print("combined wave paths: ", csv_combined_wave_path)

# use a dictionary to store the csv path and the wave path since they are different when using mounted google drive
path_dict = {}
for i in range(len(csv_combined_wave_path)):
  path_dict[csv_combined_wave_path[i]] = csv_combined_wave_path[i]
print(path_dict)

mfcc_test = []

# loop through each shout_data subfolder folder and conduct mfcc on each
# change selected_files to csv_wave_path for all wave files

mfcc_test = [extract_mfcc(path_dict[file_path]) for file_path in csv_combined_wave_path]

# can use mfcc_test to train model below
mfcc_test = np.array(mfcc_test)

print("Test MFCC size: ", len(mfcc_test), ", shape:", mfcc_test.shape, "sequence: ", mfcc_test)

np.save('mfccs_test.npy', mfcc_test)




combined wave paths:  ['/content/shout-data/shout_data_182acf98-9ee6-43ac-8200-1ba71b05ab13/final/chunk10.wav', '/content/shout-data/shout_data_28492b1e-a9ff-42fc-801f-75a79fde2051/final/chunk11.wav', '/content/shout-data/shout_data_19f32cb9-4e31-4773-a9fa-bf135f0e5008/final/chunk00.wav', '/content/shout-data/shout_data_44ec8f04-e3c7-43a7-bcd8-631d7e27c86b/final/chunk31.wav', '/content/shout-data/shout_data_8689180c-a74d-4bc5-ae0a-e38b1703b263/final/chunk10.wav', '/content/shout-data/shout_data_3fec94ec-1c01-4ae7-a274-9a57082fcf37/final/chunk10.wav', '/content/shout-data/shout_data_b1ec7e42-ba4a-4ba5-91c5-9caa270d800b/final/chunk02.wav', '/content/shout-data/shout_data_87372db0-9d15-4f91-98f0-3dd39270b8fe/final/chunk31.wav', '/content/shout-data/shout_data_a42cdded-70b7-4bcf-a0d3-312eb05c1b8b/final/chunk10.wav', '/content/shout-data/shout_data_9ef1aba0-7273-43d1-8441-9bd3d83dd701/final/chunk23.wav', '/content/shout-data/shout_data_9c43e789-eb4d-4546-92b7-1393944dff03/final/chunk01.wav'

train the model

In [8]:
# Extract MFCCs and store in separate variables
X_train = mfcc_train
X_test = mfcc_test

# Convert 'affect' labels to numerical values
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(training_data['affect'])
# y_test = label_encoder.transform(test_data['affect'])
y_test = label_encoder.transform(test_data['perceived emotion'])

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Split the training data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension

# Define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:]), kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Reshape(target_shape=(-1, 64)))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))  # Reduce dropout rate
model.add(LSTM(64))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer with number of classes

# Compile model
optimizer = Adam(learning_rate=0.001)  # Adjust learning rate as needed
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))


Epoch 1/50
37/37 [==============================] - 63s 2s/step - loss: 3.0212 - accuracy: 0.1287 - val_loss: 2.3665 - val_accuracy: 0.1303
Epoch 2/50
37/37 [==============================] - 51s 1s/step - loss: 2.1603 - accuracy: 0.1503 - val_loss: 2.0392 - val_accuracy: 0.1303
Epoch 3/50
37/37 [==============================] - 50s 1s/step - loss: 2.0013 - accuracy: 0.1634 - val_loss: 1.9822 - val_accuracy: 0.1540
Epoch 4/50
37/37 [==============================] - 50s 1s/step - loss: 1.9738 - accuracy: 0.1643 - val_loss: 1.9760 - val_accuracy: 0.1472
Epoch 5/50
37/37 [==============================] - 54s 1s/step - loss: 1.9621 - accuracy: 0.1791 - val_loss: 1.9823 - val_accuracy: 0.1303
Epoch 6/50
37/37 [==============================] - 51s 1s/step - loss: 1.9626 - accuracy: 0.1990 - val_loss: 2.0228 - val_accuracy: 0.1371
Epoch 7/50
37/37 [==============================] - 50s 1s/step - loss: 1.9955 - accuracy: 0.2087 - val_loss: 2.0485 - val_accuracy: 0.1337
Epoch 8/50
37/37 [==

In [9]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'CRNN test loss: {loss}')
print(f'CRNN test accuracy: {accuracy}')

# Convert predicted probabilities to labels
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate against 'perceived emotions'
perceived_emotions = test_data['perceived emotion']
perceived_emotions = label_encoder.transform(perceived_emotions)

# Calculate and display F1 score and accuracy
f1 = f1_score(perceived_emotions, y_pred, average='weighted')
accuracy = accuracy_score(perceived_emotions, y_pred)
print(f'CRNN F1 score on percieved emotions: {f1}')
print(f'CRNN accuracy on percieved emotions: {accuracy}')

# Display confusion matrix
conf_matrix = confusion_matrix(perceived_emotions, y_pred)
print('CRNN confusion matrix for percieved emotions:')
print(conf_matrix)

# Print classification report
print('CRNN classification report for percieved emotions:')
print(classification_report(perceived_emotions, y_pred))

4/4 [==============================] - 1s 119ms/step - loss: 2.6000 - accuracy: 0.3500
CRNN test loss: 2.599961757659912
CRNN test accuracy: 0.3499999940395355
4/4 [==============================] - 2s 114ms/step
CRNN F1 score on percieved emotions: 0.35658251243080963
CRNN accuracy on percieved emotions: 0.35
CRNN confusion matrix for percieved emotions:
[[9 9 2 2 3 1 1]
 [3 3 1 1 0 1 0]
 [0 0 8 0 0 0 1]
 [0 2 0 1 1 0 4]
 [2 4 4 1 5 3 0]
 [2 3 1 1 0 3 0]
 [1 4 2 2 2 1 6]]
CRNN classification report for percieved emotions:
              precision    recall  f1-score   support

           0       0.53      0.33      0.41        27
           1       0.12      0.33      0.18         9
           2       0.44      0.89      0.59         9
           3       0.12      0.12      0.12         8
           4       0.45      0.26      0.33        19
           5       0.33      0.30      0.32        10
           6       0.50      0.33      0.40        18

    accuracy                         